### Importing required libraries

In [1]:
import requests
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd

### Collecting web page data using selenium

In [176]:
result_list=[]
df_list=[]
driver=webdriver.Chrome()
for i in range(1,101):
    url=f'https://www.naukri.com/data-analyst-jobs-{i}?k=data%20analyst&experience=1'
    driver.get(url)
    if i==1:
        time.sleep(20)
    else:
        time.sleep(5)
        soup = BeautifulSoup(driver.page_source,'html.parser')
        result=soup.find('div',class_='list')
        df_list.append(to_data_frame(result))
driver.close()

### Function to collect required data

In [142]:
def to_data_frame(result):
    title_=[]
    rating_=[]
    company_=[]
    no_reviews_=[]
    experience_=[]
    salary_=[]
    location_=[]
    description_=[]
    skills_=[]
    from_date_=[]
    l=result.find_all('article',class_="jobTuple")
    for test in l:
        title=test.find('a', class_='title ellipsis').text if test.find('a', class_='title ellipsis') else 'None'
        if test.find('div',class_="companyInfo subheading"):
            rating=test.find('div',class_="companyInfo subheading").find(class_="starRating fleft").text if test.find('div',class_="companyInfo subheading").find(class_="starRating fleft") else 'None'
            company=test.find('div',class_="companyInfo subheading").find(class_="subTitle ellipsis fleft").text if test.find('div',class_="companyInfo subheading").find(class_="subTitle ellipsis fleft") else 'None'
            No_reviews=test.find('div',class_="companyInfo subheading").find(class_="reviewsCount fleft").text if test.find('div',class_="companyInfo subheading").find(class_="reviewsCount fleft") else 'None'
        else:
            rating,company,No_reviews=('None','None','None')
        if test.ul:
            experience=test.ul.find(class_="ellipsis fleft expwdth").text if test.ul.find(class_="ellipsis fleft expwdth") else 'None'
            salary=test.ul.find(class_="ellipsis fleft").text if test.ul.find(class_="ellipsis fleft") else 'None'
            location=test.ul.find(class_="ellipsis fleft locWdth").text if test.ul.find(class_="ellipsis fleft locWdth") else 'None'
        else:
            experience,salary,location=('None','None','None')
        description=test.find(class_="ellipsis job-description").text if test.find(class_="ellipsis job-description") else 'None'
        skill_list=[]
        if test.find('ul',class_="tags has-description"):
            skills_n=test.find('ul',class_="tags has-description").find_all('li',class_="fleft dot")
            for i in skills_n:
                skill_list.append(i.text)
        else:
            skill_list.append('None')

        release_date=test.find(class_="fleft postedDate").text if test.find(class_="fleft postedDate") else 'None'
        title_.append(title)
        rating_.append(rating)
        company_.append(company)
        no_reviews_.append(No_reviews)
        experience_.append(experience)
        salary_.append(salary)
        location_.append(location)
        description_.append(description)
        skills_.append(skill_list)
        from_date_.append(release_date)
    df=pd.DataFrame({'title':title_,'rating':rating_,'company':company_,'reviews':no_reviews_,'experience':experience_,'salary':salary_,'location':location_,'description':description_,'skills':skills_,'from_date':from_date_})
    return df

In [501]:
df=pd.concat(df_list,ignore_index=True)

## Data Cleaning

In [8]:
df.shape

(1973, 10)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1973 entries, 0 to 1972
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      1973 non-null   int64  
 1   title           1973 non-null   object 
 2   rating          1973 non-null   float64
 3   company         1973 non-null   object 
 4   reviews         1973 non-null   int64  
 5   salary          1973 non-null   int64  
 6   location        1973 non-null   object 
 7   skills          1973 non-null   object 
 8   from_date       1973 non-null   object 
 9   min_experience  1973 non-null   int64  
dtypes: float64(1), int64(4), object(5)
memory usage: 154.3+ KB


In [10]:
df.describe(include='all')

,Unnamed: 0,title,rating,company,reviews,salary,location,skills,from_date,min_experience
count,1973.000000,1973,1973.000000,1973,1973.000000,1973.000000,1973,1973,1973,1973.000000
unique,NaN,1220,NaN,1220,NaN,NaN,388,1810,32,NaN
top,NaN,Data Analyst,NaN,Bajaj Finserv Ltd.,NaN,NaN,Bangalore/Bengaluru,"('Help Desk', 'Customer Service', 'data analys...",30+ Days,NaN
freq,NaN,129,NaN,36,NaN,NaN,283,19,1196,NaN
mean,988.761277,NaN,2.733249,NaN,861.658895,28383.172833,NaN,NaN,NaN,2.011151
std,571.785808,NaN,1.828748,NaN,3427.377066,95779.449793,NaN,NaN,NaN,12.189090
min,0.000000,NaN,0.000000,NaN,0.000000,0.000000,NaN,NaN,NaN,0.000000
25%,494.000000,NaN,0.000000,NaN,0.000000,0.000000,NaN,NaN,NaN,0.000000
50%,988.000000,NaN,3.600000,NaN,12.000000,0.000000,NaN,NaN,NaN,1.000000
75%,1483.000000,NaN,4.100000,NaN,117.000000,0.000000,NaN,NaN,NaN,1.000000


In [11]:
df.columns

Index(['Unnamed: 0', 'title', 'rating', 'company', 'reviews', 'salary',
       'location', 'skills', 'from_date', 'min_experience'],
      dtype='object')

### Dealing with duplicates

In [12]:
df.skills=df.skills.apply(lambda x: tuple(x))

In [13]:
df.duplicated().sum()

0

In [14]:
df.drop_duplicates(inplace=True)

In [15]:
df.shape

(1973, 10)

### Droping description column

In [17]:
df.drop('description',axis=1,inplace=True)

### Changing the data types

In [18]:
df.loc[df['rating']=='None','rating']=0

In [19]:
df['rating']=pd.to_numeric(df['rating'])

In [20]:
df.loc[df['reviews']=='None','reviews']=0

In [21]:
df['reviews']=pd.to_numeric(df['reviews'].apply(lambda x: 0 if x==0 else x[:-8]))

TypeError: 'int' object is not subscriptable

In [ ]:
df.experience

In [ ]:
df['min_experience']=df.experience.apply(lambda x: x if type(x)!= str else (x[0] if x!='None' else 100))

In [ ]:
df.drop('experience',axis=1,inplace=True)

In [ ]:
df.salary=df.salary.apply(lambda x: x if type(x)!= str else(int(x[0])*100000 if 'PA' in x else 0))

In [ ]:
df['from_date']=df['from_date'].apply(lambda x :(x[:-4]) if len(x)>5 else x)

In [ ]:
df['from_date'].unique()

In [ ]:
df['from_date'].replace({'Few Hours':'1 Day','Just':'1 Day','Today':'1 Day'},inplace=True)

In [374]:
df.to_csv('raw_cleaned_data.csv')

In [314]:
import pandas as pd
df=pd.read_csv('raw_cleaned_data.csv')

In [315]:
df.drop('Unnamed: 0',axis=1,inplace=True)

In [316]:
df

,title,rating,company,reviews,salary,location,skills,from_date,min_experience
0,Data Analyst,3.9,Realpage,323,0,None,"('Data Validation', 'Data Analysis', 'Data Cle...",1 Day,100
1,Hiring For Data Analyst role,3.4,Estee Advisors,28,0,"Gandhinagar, Gujarat","('c++', 'Data Analytics', 'Data Science', 'fre...",23 Days,0
2,Data Analyst,3.6,Dun & Bradstreet,231,400000,"Navi Mumbai, Maharashtra","('excel', 'communication skills', 'Data', 'Dat...",7 Days,1
3,Data Analyst,4.3,Digital Green,9,0,"New Delhi, Bangalore/Bengaluru","('Data analysis', 'Automation', 'Manager Quali...",6 Days,1
4,Data Analyst,0.0,Peopleplease Consulting,0,100000,Mumbai (All Areas),"('Data Extraction', 'Data reporting', 'Python'...",1 Day,0
...,...,...,...,...,...,...,...,...,...
1968,Associate / Sr . Associate Market Research,3.8,Infiniti Research,284,0,Bengaluru,"('MS Office suite', 'Data analysis', 'Market s...",30+ Days,1
1969,Business Operations Executive,3.1,Dealsdray Online,51,300000,None,"('Advanced Excel', 'SQL', 'MS Access', 'Data A...",7 Days,100
1970,HR Exit,3.8,Trigent Software,345,0,Bangalore/Bengaluru,"('exit formalities', 'exit', 'HR exit', 'HR Ex...",30+ Days,1
1971,Transport Planner - Micro - Simulation,4.0,SYSTRA,310,0,Bangalore/Bengaluru,"('GIS', 'Data analysis', 'Visual Basic', 'Simu...",30+ Days,1


In [317]:
df.reset_index(inplace=True)
#df.rename(columns={'index': 'job id'}, inplace=True)

In [318]:
df.rename({'index':'job_id'},axis=1,inplace=True)

In [319]:
df.shape

(1973, 10)

### Spliting the location to atomic

In [320]:
len(df['location'])

1973

In [321]:
df_copy=df.copy()

In [344]:
df=df_copy

In [345]:
import re

In [346]:
location_list=df['location'].apply (lambda x : x.split(','))

In [347]:
df['location']

0                                 None
1                 Gandhinagar, Gujarat
2             Navi Mumbai, Maharashtra
3       New Delhi, Bangalore/Bengaluru
4                   Mumbai (All Areas)
                     ...              
1968                         Bengaluru
1969                              None
1970               Bangalore/Bengaluru
1971               Bangalore/Bengaluru
1972               Bangalore/Bengaluru
Name: location, Length: 1973, dtype: object

In [348]:
location_list

0                                  [None]
1                 [Gandhinagar,  Gujarat]
2             [Navi Mumbai,  Maharashtra]
3       [New Delhi,  Bangalore/Bengaluru]
4                    [Mumbai (All Areas)]
                      ...                
1968                          [Bengaluru]
1969                               [None]
1970                [Bangalore/Bengaluru]
1971                [Bangalore/Bengaluru]
1972                [Bangalore/Bengaluru]
Name: location, Length: 1973, dtype: object

In [349]:
df_location=df.loc[df.location.index.repeat(location_list.apply(len))]

In [350]:
loc_list=[]
for sublist in location_list:
    for loc in sublist:
        loc_list.append(loc)

In [351]:
df_location['location']=loc_list

In [352]:
df_location['location']=df_location['location'].str.strip()

In [353]:
df_location.reset_index(drop=True)

,job_id,title,rating,company,reviews,salary,location,skills,from_date,min_experience
0,0,Data Analyst,3.9,Realpage,323,0,None,"('Data Validation', 'Data Analysis', 'Data Cle...",1 Day,100
1,1,Hiring For Data Analyst role,3.4,Estee Advisors,28,0,Gandhinagar,"('c++', 'Data Analytics', 'Data Science', 'fre...",23 Days,0
2,1,Hiring For Data Analyst role,3.4,Estee Advisors,28,0,Gujarat,"('c++', 'Data Analytics', 'Data Science', 'fre...",23 Days,0
3,2,Data Analyst,3.6,Dun & Bradstreet,231,400000,Navi Mumbai,"('excel', 'communication skills', 'Data', 'Dat...",7 Days,1
4,2,Data Analyst,3.6,Dun & Bradstreet,231,400000,Maharashtra,"('excel', 'communication skills', 'Data', 'Dat...",7 Days,1
...,...,...,...,...,...,...,...,...,...,...
2950,1968,Associate / Sr . Associate Market Research,3.8,Infiniti Research,284,0,Bengaluru,"('MS Office suite', 'Data analysis', 'Market s...",30+ Days,1
2951,1969,Business Operations Executive,3.1,Dealsdray Online,51,300000,None,"('Advanced Excel', 'SQL', 'MS Access', 'Data A...",7 Days,100
2952,1970,HR Exit,3.8,Trigent Software,345,0,Bangalore/Bengaluru,"('exit formalities', 'exit', 'HR exit', 'HR Ex...",30+ Days,1
2953,1971,Transport Planner - Micro - Simulation,4.0,SYSTRA,310,0,Bangalore/Bengaluru,"('GIS', 'Data analysis', 'Visual Basic', 'Simu...",30+ Days,1


In [354]:
df_location.location=df_location.location.apply(lambda x: x.split('/')[0]).apply(lambda x: x.split('(')[0]).str.strip()

In [355]:
df_location

,job_id,title,rating,company,reviews,salary,location,skills,from_date,min_experience
0,0,Data Analyst,3.9,Realpage,323,0,None,"('Data Validation', 'Data Analysis', 'Data Cle...",1 Day,100
1,1,Hiring For Data Analyst role,3.4,Estee Advisors,28,0,Gandhinagar,"('c++', 'Data Analytics', 'Data Science', 'fre...",23 Days,0
1,1,Hiring For Data Analyst role,3.4,Estee Advisors,28,0,Gujarat,"('c++', 'Data Analytics', 'Data Science', 'fre...",23 Days,0
2,2,Data Analyst,3.6,Dun & Bradstreet,231,400000,Navi Mumbai,"('excel', 'communication skills', 'Data', 'Dat...",7 Days,1
2,2,Data Analyst,3.6,Dun & Bradstreet,231,400000,Maharashtra,"('excel', 'communication skills', 'Data', 'Dat...",7 Days,1
...,...,...,...,...,...,...,...,...,...,...
1968,1968,Associate / Sr . Associate Market Research,3.8,Infiniti Research,284,0,Bengaluru,"('MS Office suite', 'Data analysis', 'Market s...",30+ Days,1
1969,1969,Business Operations Executive,3.1,Dealsdray Online,51,300000,None,"('Advanced Excel', 'SQL', 'MS Access', 'Data A...",7 Days,100
1970,1970,HR Exit,3.8,Trigent Software,345,0,Bangalore,"('exit formalities', 'exit', 'HR exit', 'HR Ex...",30+ Days,1
1971,1971,Transport Planner - Micro - Simulation,4.0,SYSTRA,310,0,Bangalore,"('GIS', 'Data analysis', 'Visual Basic', 'Simu...",30+ Days,1


In [356]:
pattern=r'B[ae]ngal[ou]r[ue]'

In [357]:
values_to_replace=df_location['location'][df_location['location'].str.contains(pattern)].unique()

In [358]:
df_location['location'].replace(values_to_replace,'Bengaluru',inplace=True)

In [359]:
df_location

,job_id,title,rating,company,reviews,salary,location,skills,from_date,min_experience
0,0,Data Analyst,3.9,Realpage,323,0,None,"('Data Validation', 'Data Analysis', 'Data Cle...",1 Day,100
1,1,Hiring For Data Analyst role,3.4,Estee Advisors,28,0,Gandhinagar,"('c++', 'Data Analytics', 'Data Science', 'fre...",23 Days,0
1,1,Hiring For Data Analyst role,3.4,Estee Advisors,28,0,Gujarat,"('c++', 'Data Analytics', 'Data Science', 'fre...",23 Days,0
2,2,Data Analyst,3.6,Dun & Bradstreet,231,400000,Navi Mumbai,"('excel', 'communication skills', 'Data', 'Dat...",7 Days,1
2,2,Data Analyst,3.6,Dun & Bradstreet,231,400000,Maharashtra,"('excel', 'communication skills', 'Data', 'Dat...",7 Days,1
...,...,...,...,...,...,...,...,...,...,...
1968,1968,Associate / Sr . Associate Market Research,3.8,Infiniti Research,284,0,Bengaluru,"('MS Office suite', 'Data analysis', 'Market s...",30+ Days,1
1969,1969,Business Operations Executive,3.1,Dealsdray Online,51,300000,None,"('Advanced Excel', 'SQL', 'MS Access', 'Data A...",7 Days,100
1970,1970,HR Exit,3.8,Trigent Software,345,0,Bengaluru,"('exit formalities', 'exit', 'HR exit', 'HR Ex...",30+ Days,1
1971,1971,Transport Planner - Micro - Simulation,4.0,SYSTRA,310,0,Bengaluru,"('GIS', 'Data analysis', 'Visual Basic', 'Simu...",30+ Days,1


### droping the state names

In [360]:
df_location=df_location[~(df_location['location'].isin(["Andhra Pradesh", "Arunachal Pradesh", "Assam", "Bihar", "Chhattisgarh", "Goa", "Gujarat", "Haryana",
                                                                        "Himachal Pradesh", "Jharkhand", "Karnataka", "Kerala", "Madhya Pradesh", "Maharashtra", "Manipur", 
                                                                        "Meghalaya", "Mizoram", "Nagaland", "Odisha", "Punjab", "Rajasthan", "Sikkim", "Tamil Nadu", "Telangana", 
                                                                        "Tripura", "Uttar Pradesh", "Uttarakhand", "West Bengal"]
))]

In [361]:
df_location

,job_id,title,rating,company,reviews,salary,location,skills,from_date,min_experience
0,0,Data Analyst,3.9,Realpage,323,0,None,"('Data Validation', 'Data Analysis', 'Data Cle...",1 Day,100
1,1,Hiring For Data Analyst role,3.4,Estee Advisors,28,0,Gandhinagar,"('c++', 'Data Analytics', 'Data Science', 'fre...",23 Days,0
2,2,Data Analyst,3.6,Dun & Bradstreet,231,400000,Navi Mumbai,"('excel', 'communication skills', 'Data', 'Dat...",7 Days,1
3,3,Data Analyst,4.3,Digital Green,9,0,New Delhi,"('Data analysis', 'Automation', 'Manager Quali...",6 Days,1
3,3,Data Analyst,4.3,Digital Green,9,0,Bengaluru,"('Data analysis', 'Automation', 'Manager Quali...",6 Days,1
...,...,...,...,...,...,...,...,...,...,...
1968,1968,Associate / Sr . Associate Market Research,3.8,Infiniti Research,284,0,Bengaluru,"('MS Office suite', 'Data analysis', 'Market s...",30+ Days,1
1969,1969,Business Operations Executive,3.1,Dealsdray Online,51,300000,None,"('Advanced Excel', 'SQL', 'MS Access', 'Data A...",7 Days,100
1970,1970,HR Exit,3.8,Trigent Software,345,0,Bengaluru,"('exit formalities', 'exit', 'HR exit', 'HR Ex...",30+ Days,1
1971,1971,Transport Planner - Micro - Simulation,4.0,SYSTRA,310,0,Bengaluru,"('GIS', 'Data analysis', 'Visual Basic', 'Simu...",30+ Days,1


In [363]:
df_location

,job_id,title,rating,company,reviews,salary,location,skills,from_date,min_experience
0,0,Data Analyst,3.9,Realpage,323,0,None,"('Data Validation', 'Data Analysis', 'Data Cle...",1 Day,100
1,1,Hiring For Data Analyst role,3.4,Estee Advisors,28,0,Gandhinagar,"('c++', 'Data Analytics', 'Data Science', 'fre...",23 Days,0
2,2,Data Analyst,3.6,Dun & Bradstreet,231,400000,Navi Mumbai,"('excel', 'communication skills', 'Data', 'Dat...",7 Days,1
3,3,Data Analyst,4.3,Digital Green,9,0,New Delhi,"('Data analysis', 'Automation', 'Manager Quali...",6 Days,1
3,3,Data Analyst,4.3,Digital Green,9,0,Bengaluru,"('Data analysis', 'Automation', 'Manager Quali...",6 Days,1
...,...,...,...,...,...,...,...,...,...,...
1968,1968,Associate / Sr . Associate Market Research,3.8,Infiniti Research,284,0,Bengaluru,"('MS Office suite', 'Data analysis', 'Market s...",30+ Days,1
1969,1969,Business Operations Executive,3.1,Dealsdray Online,51,300000,None,"('Advanced Excel', 'SQL', 'MS Access', 'Data A...",7 Days,100
1970,1970,HR Exit,3.8,Trigent Software,345,0,Bengaluru,"('exit formalities', 'exit', 'HR exit', 'HR Ex...",30+ Days,1
1971,1971,Transport Planner - Micro - Simulation,4.0,SYSTRA,310,0,Bengaluru,"('GIS', 'Data analysis', 'Visual Basic', 'Simu...",30+ Days,1


In [365]:
df_location.head()

,job_id,title,rating,company,reviews,salary,location,skills,from_date,min_experience
0,0,Data Analyst,3.9,Realpage,323,0,None,"('Data Validation', 'Data Analysis', 'Data Cle...",1 Day,100
1,1,Hiring For Data Analyst role,3.4,Estee Advisors,28,0,Gandhinagar,"('c++', 'Data Analytics', 'Data Science', 'fre...",23 Days,0
2,2,Data Analyst,3.6,Dun & Bradstreet,231,400000,Navi Mumbai,"('excel', 'communication skills', 'Data', 'Dat...",7 Days,1
3,3,Data Analyst,4.3,Digital Green,9,0,New Delhi,"('Data analysis', 'Automation', 'Manager Quali...",6 Days,1
3,3,Data Analyst,4.3,Digital Green,9,0,Bengaluru,"('Data analysis', 'Automation', 'Manager Quali...",6 Days,1


In [366]:
df_location.location.value_counts()[(df_location.location.value_counts()>20)]

Bengaluru    448
Mumbai       288
Hyderabad    237
Pune         197
Gurgaon      175
Chennai      173
New Delhi    112
Remote        97
Noida         95
Kolkata       90
Ahmedabad     88
Delhi         84
Surat         29
Mohali        25
Kochi         24
Lucknow       23
Vadodara      22
Indore        21
Jaipur        21
Name: location, dtype: int64

In [367]:
df_location_copy=df_location.copy()

#### Creating a new csv file with job id and locations

In [368]:
loc_table=df_location[['job_id','location']]

In [369]:
loc_table.to_csv('loc_table.csv',index=False)

In [246]:
#df_location.drop('Unnamed: 0',axis=1,inplace=True)

In [67]:
df_location.to_csv('location_cleaned_data.csv',index=False)

In [313]:
df_location.head()

,job_id,title,rating,company,reviews,salary,location,skills,from_date,min_experience
0,0,Data Analyst,3.9,Realpage,323,0,None,"('Data Validation', 'Data Analysis', 'Data Cle...",1 Day,100
1,1,Hiring For Data Analyst role,3.4,Estee Advisors,28,0,None,"('c++', 'Data Analytics', 'Data Science', 'fre...",23 Days,0
2,2,Data Analyst,3.6,Dun & Bradstreet,231,400000,None,"('excel', 'communication skills', 'Data', 'Dat...",7 Days,1
3,3,Data Analyst,4.3,Digital Green,9,0,None,"('Data analysis', 'Automation', 'Manager Quali...",6 Days,1
3,3,Data Analyst,4.3,Digital Green,9,0,None,"('Data analysis', 'Automation', 'Manager Quali...",6 Days,1


### Split skills to individual raws

In [253]:
df.shape

(1973, 10)

In [254]:
df_skills=df.copy()

In [269]:
df_skills['skills']=df_skills['skills'].apply(lambda x: x[1:-1].split(','))

In [270]:
skill_individual_list=[]
count=0
for skilltuple in df_skills['skills']:
    for skill in skilltuple:
        skill_individual_list.append(skill)
len(skill_individual_list)

15059

In [272]:
df_skills=df_skills.loc[df_skills.skills.index.repeat(df_skills.skills.apply(len))]

In [273]:
print(len(df_skills))
print(len(skill_individual_list))

15059
15059


In [274]:
df_skills['skills']=skill_individual_list

In [277]:
df_skills.columns

Index(['job_id', 'title', 'rating', 'company', 'reviews', 'salary', 'location',
       'skills', 'from_date', 'min_experience'],
      dtype='object')

In [283]:
df_skills.reset_index(drop=True,inplace=True)

In [115]:
df_skills.to_csv('skills_cleaned_data.csv')

In [69]:
#df_skills=pd.read_csv('skills_cleaned_data.csv')

In [71]:
df_skills['skills'].unique()

array(['Data Validation', 'Data Analysis', 'Data Cleansing', ...,
       'Planning engineering', 'Engineering project management',
       'Civil engineering'], dtype=object)

In [288]:
df_skills['skills'].isna().sum()

0

In [94]:
df_skills.dropna(inplace=True)

In [289]:
df_skills.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15059 entries, 0 to 15058
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   job_id          15059 non-null  int64  
 1   title           15059 non-null  object 
 2   rating          15059 non-null  float64
 3   company         15059 non-null  object 
 4   reviews         15059 non-null  int64  
 5   salary          15059 non-null  int64  
 6   location        15059 non-null  object 
 7   skills          15059 non-null  object 
 8   from_date       15059 non-null  object 
 9   min_experience  15059 non-null  int64  
dtypes: float64(1), int64(4), object(5)
memory usage: 1.1+ MB


In [290]:
pattern2=r'[Aa]naly[s]?[ti]?s?'
alalyst_skills=df_skills['skills'].loc[df_skills['skills'].str.contains(pattern2)].unique()

In [291]:
df_skills['skills'].replace(alalyst_skills,'Analytics skills',inplace=True)

In [292]:
df_skills['skills'].value_counts()[df_skills['skills'].value_counts()>40]

Analytics skills                2529
 'Excel'                         156
 'Machine learning'              145
 'SQL'                           137
 'Management'                    120
 'MIS'                           111
 'Data management'               106
 'Data'                          105
 'Data quality'                   95
 'Project management'             88
 'Healthcare'                     85
 'Market research'                83
 'Operations'                     83
 'Consulting'                     83
 'Research'                       83
 'MS Office'                      79
 'Python'                         78
 'Data mining'                    77
 'Data collection'                75
 'Data modeling'                  73
 'Business intelligence'          73
 'Social media'                   70
 'data'                           65
 'Advanced Excel'                 65
'Training'                        63
 'Sales'                          61
 'Agile'                          61
 

In [293]:
df_skills['skills'].replace({'Excel':'Advanced Excel','VLOOKUP':'Advanced Excel'},inplace=True)

In [294]:
skills_table=df_skills[['job_id','skills']]

In [308]:
skills_table['skills']=skills_table['skills'].str.strip().apply(lambda x: x[1:-1] if "'" in x else x)

C:\Users\USER\AppData\Local\Temp\ipykernel_11648\1244037516.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  skills_table['skills']=skills_table['skills'].str.strip().apply(lambda x: x[1:-1] if "'" in x else x)


In [309]:
skills_table.to_csv('skills_table.csv',index=False)

### Joining the data frames

In [298]:
df.head()

,job_id,title,rating,company,reviews,salary,location,skills,from_date,min_experience
0,0,Data Analyst,3.9,Realpage,323,0,None,"('Data Validation', 'Data Analysis', 'Data Cle...",1 Day,100
1,1,Hiring For Data Analyst role,3.4,Estee Advisors,28,0,"Gandhinagar, Gujarat","('c++', 'Data Analytics', 'Data Science', 'fre...",23 Days,0
2,2,Data Analyst,3.6,Dun & Bradstreet,231,400000,"Navi Mumbai, Maharashtra","('excel', 'communication skills', 'Data', 'Dat...",7 Days,1
3,3,Data Analyst,4.3,Digital Green,9,0,"New Delhi, Bangalore/Bengaluru","('Data analysis', 'Automation', 'Manager Quali...",6 Days,1
4,4,Data Analyst,0.0,Peopleplease Consulting,0,100000,Mumbai (All Areas),"('Data Extraction', 'Data reporting', 'Python'...",1 Day,0


In [373]:
loc_table.head()

,job_id,location
0,0,None
1,1,Gandhinagar
2,2,Navi Mumbai
3,3,New Delhi
3,3,Bengaluru


In [310]:
skills_table

,job_id,skills
0,0,Data Validation
1,0,Analytics skills
2,0,Data Cleansing
3,0,Data Collection
4,0,Excel
...,...,...
15054,1972,Simulation
15055,1972,Urban planning
15056,1972,Consulting
15057,1972,Operations


In [376]:
pd.merge(df,loc_table,on='job_id')

,job_id,title,rating,company,reviews,salary,location_x,skills,from_date,min_experience,location_y
0,0,Data Analyst,3.9,Realpage,323,0,None,"('Data Validation', 'Data Analysis', 'Data Cle...",1 Day,100,None
1,1,Hiring For Data Analyst role,3.4,Estee Advisors,28,0,"Gandhinagar, Gujarat","('c++', 'Data Analytics', 'Data Science', 'fre...",23 Days,0,Gandhinagar
2,2,Data Analyst,3.6,Dun & Bradstreet,231,400000,"Navi Mumbai, Maharashtra","('excel', 'communication skills', 'Data', 'Dat...",7 Days,1,Navi Mumbai
3,3,Data Analyst,4.3,Digital Green,9,0,"New Delhi, Bangalore/Bengaluru","('Data analysis', 'Automation', 'Manager Quali...",6 Days,1,New Delhi
4,3,Data Analyst,4.3,Digital Green,9,0,"New Delhi, Bangalore/Bengaluru","('Data analysis', 'Automation', 'Manager Quali...",6 Days,1,Bengaluru
...,...,...,...,...,...,...,...,...,...,...,...
2725,1968,Associate / Sr . Associate Market Research,3.8,Infiniti Research,284,0,Bengaluru,"('MS Office suite', 'Data analysis', 'Market s...",30+ Days,1,Bengaluru
2726,1969,Business Operations Executive,3.1,Dealsdray Online,51,300000,None,"('Advanced Excel', 'SQL', 'MS Access', 'Data A...",7 Days,100,None
2727,1970,HR Exit,3.8,Trigent Software,345,0,Bangalore/Bengaluru,"('exit formalities', 'exit', 'HR exit', 'HR Ex...",30+ Days,1,Bengaluru
2728,1971,Transport Planner - Micro - Simulation,4.0,SYSTRA,310,0,Bangalore/Bengaluru,"('GIS', 'Data analysis', 'Visual Basic', 'Simu...",30+ Days,1,Bengaluru


In [378]:
merged_df=pd.merge(pd.merge(df,loc_table,on='job_id'),skills_table,on='job_id')

In [379]:
merged_df.drop(['location_x','skills_x'],axis=1,inplace=True)

In [380]:
merged_df

,job_id,title,rating,company,reviews,salary,from_date,min_experience,location_y,skills_y
0,0,Data Analyst,3.9,Realpage,323,0,1 Day,100,None,Data Validation
1,0,Data Analyst,3.9,Realpage,323,0,1 Day,100,None,Analytics skills
2,0,Data Analyst,3.9,Realpage,323,0,1 Day,100,None,Data Cleansing
3,0,Data Analyst,3.9,Realpage,323,0,1 Day,100,None,Data Collection
4,0,Data Analyst,3.9,Realpage,323,0,1 Day,100,None,Excel
...,...,...,...,...,...,...,...,...,...,...
20849,1972,Assistant Transport Planner - Micro-Simulation,4.0,SYSTRA,310,0,30+ Days,1,Bengaluru,Simulation
20850,1972,Assistant Transport Planner - Micro-Simulation,4.0,SYSTRA,310,0,30+ Days,1,Bengaluru,Urban planning
20851,1972,Assistant Transport Planner - Micro-Simulation,4.0,SYSTRA,310,0,30+ Days,1,Bengaluru,Consulting
20852,1972,Assistant Transport Planner - Micro-Simulation,4.0,SYSTRA,310,0,30+ Days,1,Bengaluru,Operations


In [381]:
merged_df.to_csv('merged_data.csv',index=False)